In [2]:
# AF_INET refers to the address family ipv4
# SOCK_STREAM means connection oriented TCP protocol
from socket import AF_INET,socket,SOCK_STREAM
from threading import Thread

In [3]:
clients = {}
addresses = {}
HOST = ""
Port = 33000
BUFSIZ = 1024
ADDR = (HOST,Port)
SERVER = socket(AF_INET,SOCK_STREAM)
SERVER.bind(ADDR)

In [5]:
def accept_incoming_connections():
    """Sets up handling for incoming clients"""
    while True:
        client,client_address = SERVER.accept()
        print("%s:%s has connected."%client_address)
        client.send(bytes("Greetings from the cave!"+"Now type your name and enter","utf8"))
        addresses[client]=client_address
        Thread(target=handle_client, args=(client,)).start()

In [6]:
def handle_client(client):#takes client socket as an arguement
    """Handles a single client connection"""
    name = client.recv(BUFSIZ).decode("utf8")
    welcome = "Welcome %s! If you ever want to quit, type {quit} to exit." %name
    client.send(bytes(welcome,"utf8"))
    msg = "%s has joined the chat!" %name
    broadcast(bytes(msg,"utf8"))
    clients[client] = name
    while True:
        msg = client.recv(BUFSIZ)
        if msg != bytes("{quit}","utf8"):
            broadcast(msg,name+": ")
        else:
            client.send(bytes("{quit}","utf8"))
            client.close()
            del clients[client]
            broadcast(bytes('%s had left the chat.'%name,"utf8"))
            break
        
    

In [7]:
def broadcast(msg,prefix=""): # prefix is for name identification
    """Broadcasts a message to all the clients"""
    for sock in clients:
        sock.send(bytes(prefix, "utf8")+msg)

In [ ]:
if __name__ == "__main__":
    SERVER.listen(5) # Listens for 5 connections at max
    print("Waiting for connection...")
    ACCEPT_THREAD = Thread(target=accept_incoming_connections)
    ACCEPT_THREAD.start()
    ACCEPT_THREAD.join()
    SERVER.close()

Waiting for connection...
